In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [6]:
df = pd.read_csv("MediaSentimentData.csv")

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def download_html(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to fetch {url}, status code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

# Add HTML content column to the DataFrame
df['html_content'] = df['url'].apply(download_html)

Failed to fetch https://www.the-independent.com/news/world/americas/us-politics/donald-trump-news-cabinet-picks-live-updates-b2649022.html, status code: 405
Failed to fetch https://www.poynter.org/reporting-editing/2024/biden-80-million-votes-kamala-60-million-stolen-election/, status code: 403
Failed to fetch https://viewfromthewing.com/ex-reality-tv-star-congressman-sean-duffy-chosen-as-transportation-secretary-what-it-means-for-airline-mergers-and-regulation/, status code: 403
Failed to fetch http://skift.com/2024/11/18/trump-taps-former-rep-sean-duffy-for-transportation-secreatary/, status code: 403
Failed to fetch https://viewfromthewing.com/heres-what-we-know-a-trump-administration-will-mean-for-travel/, status code: 403
Failed to fetch http://skift.com/2024/11/04/election-2024-the-issues-that-matter-for-travel/, status code: 403
Failed to fetch https://liveandletsfly.com/trump-victory-airlines/, status code: 403
Failed to fetch https://www.newsweek.com/donald-trump-ally-suggests

In [15]:
def extract_article_content(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        # Identify the main content section. This might vary depending on the website's structure.
        # Common tags/classes used for articles:
        article = soup.find('article')  # Look for <article> tag
        if not article:
            # Fallback to other heuristics if <article> tag isn't available
            article = soup.find('div', {'class': 'main-content'}) or soup.find('div', {'class': 'content'})

        # Extract text content
        return article.get_text(strip=True) if article else "Content not found"
    except Exception as e:
        print(f"Error parsing HTML: {e}")
        return "Parsing error"

# Add extracted content column
df['article_content'] = df['html_content'].apply(lambda html: extract_article_content(html) if html else "No HTML")


In [21]:
df[df["article_content"]=="Content not found"].shape

(269, 11)

In [32]:
df = df[~(df["article_content"]=="Content not found")].drop("Unnamed: 0", axis = 1).reset_index(drop = True)

In [33]:
df.to_csv("MediaSentimentData_Processed.csv")